# Simulation for Potts model
- All levels of interaction
- Field (constant for each point)
- Preference for low number of colors
- Distance between different colors (0 if equal, 1 of different)

- $N$ vertices
- $P(\sigma) = \frac{1}{N^N} \exp((N-n(\sigma))\gamma$), where $n(\sigma) = \text{# of colors used in } \sigma$

- Consider the interaction on the largest hyperedge on lattice $\Lambda$, where $|\Lambda| = N$
- We assign $\eta_{\Lambda} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with $\{1,2,...,N\}$ (a set with $k$ colors corresponds to $k$).
- Each of these corresponds to an energy level in $\{(N-1)\gamma, (N-2)\gamma, ..., 0\}$


- Consider the interaction between adjacent sites (edges) $b$ with size $|b|=2$
- To each edge assign $\eta_{b} \in \mathcal{P}(\{0,1\}^2)$ which is in one-to-one correspondence with the edge being open or closed.
- Each of these corresponds to an energy level in $\{J, -J\}$

- Consider the interaction between adjacent sites and the field, corresponding to $v$ with size $|v|=1$
- To each site assign $\eta_{v} \in \mathcal{P}(\{1,2,...,q\}^N)$ which is in one-to-one correspondence with the color of the site.
- Each of these corresponds to an energy level in $\{(N-1)\alpha, (N-2)\alpha, ...,(N-k)\alpha, ...,0\}$ where $k$ corresponds to the $k$th preferred color of the field.

- Take $ \upsilon_k =  \frac{e^{((N-k)\gamma)} - e^{((N-k-1)\gamma)}}{e^{((N-1)\gamma)}} $

- Joint distribution
- $Q(\eta, \sigma) = \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} $

- Marginals
- $Q(\eta | \sigma) =  \frac { \upsilon_{\eta_{\Lambda}} \frac{1}{N^N} \mathbb{1}_{(n(\sigma) \leq \eta_{\Lambda})} }{P(\sigma)}$
- $Q(\sigma | \eta) = \frac{1}{|\{\sigma:  \eta(\sigma) \leq \eta(\Lambda)\}|} $

In [55]:
# Initial code from https://rajeshrinet.github.io/blog/2014/ising-model/

import numpy as np 
from numpy.random import rand
from scipy.special import binom
from unionfind import UnionFind

# Implement Union Find to find clusters in the configurations
# https://github.com/deehzee/unionfind/blob/master/unionfind.py

def initial_config(N, no_colors=2):   
    ''' Generate a random color/spin configuration for initialization'''
    state = np.random.randint(low=1, high=no_colors+1, size=(N,N))
    return state


def mc_move(config, eta_prob, eta, N, no_colors, sites):
    '''Monte Carlo move using generalized SW algorithm '''
    # Assign eta to each hyperedge
    eta = assign_etas(config, eta_prob, eta, no_colors)
    # print('assigned eta:', eta)

    # Assign labels to each site
    config = assign_labels(config, eta, N, no_colors, sites)
    # print('assigned configuration:\n', config)

    return config


def number_of_colors (config):
    '''Number of colors used in the given confuration of labels'''
    return len(np.unique(config))


def avg_sites_per_color (config):
    '''Average number of sites each present color has in the configuration'''
    unq = np.unique(config)
    return config.shape[0]*config.shape[1]/len(unq)


def prob_eta_lambda(gamma, no_colors, k):
    '''Probability of the eta lambda corresponding to a certain number of colors'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*gamma) - np.exp((no_colors-k-1)*gamma) ) / np.exp((no_colors-1)*gamma)
    elif k == no_colors:
        return 1 / np.exp((no_colors-1)*gamma)
    
    
def prob_eta_edge(J, col1, col2):
    '''Probability of the eta edge corresponding to open or not'''
    if col1==col2:
        return 1 - np.exp(-2*J)
    else:
        return np.exp(-2*J)
    
    
def prob_eta_site(alpha, no_colors, k):
    '''Probability of the eta site corresponding to a maximum color number (lower have higher energy)'''
    if k < no_colors:
        return ( np.exp((no_colors-k)*alpha) - np.exp((no_colors-k-1)*alpha) ) / np.exp((no_colors-1)*alpha)
    elif k == no_colors:
        return 1 / np.exp((no_colors-1)*alpha)

    
def assign_etas(config, eta_prob, eta, no_colors):
    
    '''Assign a number of colors that is at least the current number of colors'''
    eta_lambda = eta[0]
    prob_lambda = eta_prob[0]
    print('probabilities for no. of colors (lambda):', prob_lambda)
    
    current_k = number_of_colors(config)
    p = prob_lambda[current_k-1:]
    print('current no. cols (lambda):', current_k)
    p = p / p.sum()
    eta_lambda = np.random.choice((np.arange(current_k, no_colors+1)), p=p)
    eta[0] = eta_lambda
    print('assigned no. cols (eta_lambda):', eta_lambda)

    '''Assign a closed or open bond to each edge'''
    eta_edges = eta[1]
    prob_edge = eta_prob[1]
    print('probabilities for edges:', prob_edge)
    
    # loop over edges
    for i in range(eta_edges.shape[0]):
        for j in range(eta_edges.shape[1]):
            for e in range(eta_edges.shape[2]):
                if eta_edges[i,j,e] == -1: continue
                if e==0: current_b = (0 if config[i,j]==config[i+1,j] else 1)          
                elif e==1: current_b = (0 if config[i,j]==config[i,j+1] else 1) 
                    
                p = prob_edge[current_b:]
                p = p / p.sum()
                eta_edge = np.random.choice((np.arange(current_b, 1+1)), p=p) # 0 for present bonds, 1 for not
                eta_edges[i,j,e] = eta_edge
    eta[1] = eta_edges
    print('assigned bonds (eta_edges):')
    print(eta_edges[:,:,0])
    print()
    print(eta_edges[:,:,1])
    
    '''Assign a color that has at most the current energy for the color (color number is at least the current one)'''
    eta_sites = eta[2]
    prob_site = eta_prob[2]
    print('probabilities for colors (sites):', prob_site)
    
    # loop over sites
    for i in range(eta_sites.shape[0]):
        for j in range(eta_sites.shape[1]):
            current_s = config[i,j]
            p = prob_site[current_s-1:]
            p = p / p.sum()
            # print('normalized:', p)
            eta_site = np.random.choice((np.arange(current_s, no_colors+1)), p=p)
            eta_sites[i,j] = eta_site
    eta[2] = eta_sites
    print('assigned max colors (eta_sites):')
    print(eta_sites)
    
    return eta


def site2str(tup):
    return str(tup[0]) + ',' + str(tup[1])

def str2site(strr):
    t = strr.split(',')
    return [int(x) for x in t]

def sample_config(config, eta, N, no_colors, sites):
    
    '''Generate clusters from the assigned bonds (eta_edge)'''
    uf = UnionFind(sites)
    eta_edges = eta[1]
    for i in range(eta_edges.shape[0]):
        for j in range(eta_edges.shape[1]):
            for e in range(eta_edges.shape[2]):
                if eta_edges[i,j,e] == -1: continue
                if eta_edges[i,j,e] == 0:
                    if e==0: uf.union(site2str((i,j)), site2str((i+1,j)))
                    elif e==1: uf.union(site2str((i,j)), site2str((i,j+1)))    

    '''For each cluster, find the site with strongest constraint (smallest eta_site)
       and assign that eta_site to the entire cluster
    '''
    eta_sites = eta[2]
    comp_constraints = {}
    cl_n = 0
    cls = np.zeros((eta_sites.shape[0],eta_sites.shape[1]), dtype=np.int8)
    for comp in uf.components():
        cl_n += 1
        min_constraint = no_colors
        comp_root = '-1,-1'
        for site_str in comp:
            site = str2site(site_str)
            if eta_sites[site[0],site[1]] <= min_constraint:
                min_constraint = eta_sites[site[0],site[1]]
                comp_root = site_str
            cls[site[0],site[1]] = cl_n
        comp_constraints[comp_root] = min_constraint
    print('clusters formed by bonds (eta_edge):')
    print(cls)
    
    '''Randomly sample a color for each cluster'''
    for root in comp_constraints:
        max_col = comp_constraints[root]
        comp_color = np.random.choice((np.arange(1, max_col+1)))
        for site_str in uf.component(root):
            site = str2site(site_str)
            config[site[0],site[1]] = comp_color
            
    return config

    
def assign_labels(config, eta, N, no_colors, sites):
    '''Assign a color configuration chosen uniformly from the configurations compatible with eta'''
    # Brute force version
    max_colors = eta[0] # eta_lambda
    config = sample_config(config, eta, N, no_colors, sites)
    while number_of_colors(config) > max_colors:
        config = sample_config(config, eta, N, no_colors, sites)
    return config


In [57]:
# Small experiment

N, q = 8, 6
gamma = 0.2  # strength of preference for low no. of cols
J = 0.5      # strength of preference for bonds
alpha = 0.2  # strength of preference for colors (field)

'''Probabilities ordered from highest (high energy) to lowest'''
lambda_prob = np.zeros(q)
for j in range(q):
    lambda_prob[j] = prob_eta_lambda(gamma,q,j+1)
print('lambda probabilities:', lambda_prob)

edge_prob = np.zeros(2)
edge_prob[0] = prob_eta_edge(J, 'same_col', 'same_col')
edge_prob[1] = prob_eta_edge(J, 'same_col', 'dif_col')
print('edge probabilities:', edge_prob)

site_prob = np.zeros(q)
for j in range(q):
    site_prob[j] = prob_eta_site(alpha,q,j+1)
print('site probabilities:', site_prob)

eta_prob = (lambda_prob, edge_prob, site_prob)

'''Current states of eta'''
eta_lambda = 0
eta_edges = np.zeros((N,N,2), dtype=np.int8)
# Special edge cases (no neighbors at the border)
eta_edges[:,N-1,1] = -1
eta_edges[N-1,:,0] = -1
eta_sites = np.zeros((N,N), dtype=np.int8)
eta = [eta_lambda, eta_edges, eta_sites]


'''List of sites (tuples) for Union-Find'''
sites = []
for i in range(N):
    for j in range(N):
        sites.append(str(i)+','+str(j))

config = initial_config(N,q)
print('\ninitial config:')
print(config)

# config = mc_move(config, eta_prob, eta, N, q, sites)
# print('\nnext config:')
# print(config)

for i in range(100):
    print('\n' + '-'*20 + ' iter ' + str(i) + ' ' + '-'*20)
    config = mc_move(config, eta_prob, eta, N, q, sites)
    print('config:')
    print(config)
#     if i%100 == 0: print(i)


# print('\nfinal config:')
# print(config)
# print(eta_edges[:,:,0])

lambda probabilities: [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
edge probabilities: [0.63212056 0.36787944]
site probabilities: [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]

initial config:
[[3 2 3 6 4 2 6 6]
 [5 6 1 6 4 4 4 3]
 [1 4 4 1 3 6 3 6]
 [4 1 4 1 5 1 2 4]
 [5 2 1 4 1 5 1 6]
 [6 1 1 3 5 1 4 3]
 [2 6 1 4 5 3 4 5]
 [5 4 2 6 6 3 2 3]]

-------------------- iter 0 --------------------
probabilities for no. of colors (lambda): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
current no. cols (lambda): 6
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 1  1  1  0  1  1  1  1]
 [ 1  1  1  1  1  1  1  1]
 [ 1  1  0  0  1  1  1  1]
 [ 1  1  1  1  1  1  1  1]
 [ 1  1  0  1  1  1  1  1]
 [ 1  1  1  1  0  1  0  1]
 [ 1  1  1  1  1  0  1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  1  1  1  1  1 -1]
 [ 1  1  1  1  0  0  1 -1]
 [ 1  1  1  1  1  1  1 -1]
 [ 1  1  

 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  1  0  0  0  1 -1]
 [ 0  0  0  0  0  1  1 -1]
 [ 0  1  1  1  0  0  0 -1]
 [ 0  0  1  1  0  0  0 -1]
 [ 1  0  1  0  0  1  1 -1]
 [ 1  1  1  0  1  1  0 -1]
 [ 0  0  0  1  1  0  1 -1]
 [ 1  1  1  1  0  1  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
assigned max colors (eta_sites):
[[5 3 5 6 6 4 2 6]
 [3 1 6 2 2 6 1 6]
 [5 6 6 4 6 1 2 4]
 [6 6 5 1 6 6 1 1]
 [6 6 2 6 3 1 6 6]
 [2 2 1 4 6 1 6 6]
 [3 2 3 1 5 6 4 1]
 [6 6 6 5 1 6 3 2]]
clusters formed by bonds (eta_edge):
[[ 1  3  2  3  3  3  3  3]
 [ 3  3  3  3  3  3  3  3]
 [ 6  6  7  3  3  3  3  3]
 [ 9  9  9  3  3  3  3  3]
 [ 9  9  9  3  3  3  4  3]
 [ 9  9  9  3  3  3  5  5]
 [ 9  9  9  9  3  5  5  8]
 [ 9 10 11 12  3  3 13  8]]
config:
[[1 1 3 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 3 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 3 1]
 [1 1 1 1 1 1 2 2]
 [1 1 1 1 1 2 2 1]
 [1 5 6 3 1 1 1 1]]

-------------------- iter 12 --------------------


config:
[[2 2 5 1 1 1 2 2]
 [1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 1]
 [2 1 1 1 1 1 1 1]
 [1 2 1 1 1 1 1 1]
 [5 2 2 1 1 1 1 1]
 [5 2 2 1 1 1 1 1]
 [4 1 3 1 1 1 1 2]]

-------------------- iter 22 --------------------
probabilities for no. of colors (lambda): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
current no. cols (lambda): 5
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 1  1  1  0  0  0  1  1]
 [ 0  0  0  0  1  1  0  1]
 [ 1  0  0  0  1  0  0  0]
 [ 1  1  0  0  0  1  1  0]
 [ 1  1  1  1  0  0  0  1]
 [ 0  0  1  1  0  0  0  0]
 [ 1  1  1  0  1  0  0  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 0  1  1  1  0  1  0 -1]
 [ 0  0  0  0  0  1  1 -1]
 [ 1  0  0  0  0  1  0 -1]
 [ 1  0  0  1  1  0  0 -1]
 [ 1  1  0  0  0  1  0 -1]
 [ 1  0  1  0  0  1  0 -1]
 [ 1  1  1  0  0  1  1 -1]
 [ 1  1  1  0  0  1  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
a

 [6 1 3 6 4 5 6 6]]
clusters formed by bonds (eta_edge):
[[ 1  2  2  3  4  5  6  6]
 [ 1  6  6  7  4  5  6  6]
 [ 8  6  6 10 11  6  6  6]
 [ 8  6  6  6  6  6  6  6]
 [ 6  6  6  6  6  6  6  6]
 [ 6  6  6  6  6  6  6  6]
 [ 9  9  6 12  6  6 13  6]
 [ 6  6  6 14  6  6  6  6]]
config:
[[1 3 3 4 1 2 1 1]
 [1 1 1 1 1 2 1 1]
 [1 1 1 3 2 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]
 [1 1 1 2 1 1 6 1]
 [1 1 1 6 1 1 1 1]]

-------------------- iter 35 --------------------
probabilities for no. of colors (lambda): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
current no. cols (lambda): 5
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 0  1  1  1  0  0  1  0]
 [ 0  0  0  1  1  1  0  0]
 [ 1  0  1  1  1  1  0  1]
 [ 1  0  0  0  0  1  1  0]
 [ 1  1  1  1  0  1  1  0]
 [ 0  1  0  1  0  1  1  1]
 [ 0  0  0  1  0  0  1  0]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  0  1  1  1  1  1 -1]
 [ 0  0  0  0  1  1

probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 0  1  0  0  1  1  1  1]
 [ 1  1  1  0  1  0  1  1]
 [ 1  0  0  0  1  0  1  0]
 [ 0  0  0  1  1  1  1  1]
 [ 0  0  0  0  1  1  1  1]
 [ 1  1  0  1  1  0  0  0]
 [ 0  0  1  1  0  0  1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  0  1  1  1  0 -1]
 [ 1  0  1  1  0  1  1 -1]
 [ 1  1  0  0  0  0  1 -1]
 [ 0  0  0  0  0  0  0 -1]
 [ 0  0  0  1  1  1  0 -1]
 [ 1  1  1  0  0  0  0 -1]
 [ 1  1  1  1  0  0  0 -1]
 [ 1  1  0  1  0  1  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
assigned max colors (eta_sites):
[[6 6 6 1 2 6 4 1]
 [6 1 2 1 6 6 6 6]
 [6 6 6 2 1 6 2 6]
 [5 2 1 6 6 1 2 1]
 [6 6 1 6 6 5 4 6]
 [2 6 2 6 2 3 6 1]
 [2 6 2 4 6 4 2 6]
 [6 2 6 6 2 5 5 3]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  5  6  6]
 [ 1  3  3  3  3  3 10 11]
 [12  3  3  3  3  3  3  3]
 [ 3  3  3  3  3  3  3  3]
 [ 3  3  3  3  7  8  9  9]
 [ 3  3  3  3  3  3  3  3]
 

 [1 3 6 2 5 6 4 2]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  5  6  6]
 [ 7  8  9 10 13 11 12  6]
 [13 13 13 13 13 14 12 12]
 [15 16 17 13 13 13 12 12]
 [18 19 20 21 22 23 12 12]
 [24 25 12 12 12 12 12 12]
 [25 25 12 12 12 12 12 12]
 [25 25 26 27 28 29 30 30]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  5  6  6]
 [ 7  8  9 10 13 11 12  6]
 [13 13 13 13 13 14 12 12]
 [15 16 17 13 13 13 12 12]
 [18 19 20 21 22 23 12 12]
 [24 25 12 12 12 12 12 12]
 [25 25 12 12 12 12 12 12]
 [25 25 26 27 28 29 30 30]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  5  6  6]
 [ 7  8  9 10 13 11 12  6]
 [13 13 13 13 13 14 12 12]
 [15 16 17 13 13 13 12 12]
 [18 19 20 21 22 23 12 12]
 [24 25 12 12 12 12 12 12]
 [25 25 12 12 12 12 12 12]
 [25 25 26 27 28 29 30 30]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  5  6  6]
 [ 7  8  9 10 13 11 12  6]
 [13 13 13 13 13 14 12 12]
 [15 16 17 13 13 13 12 12]
 [18 19 20 21 22 23 12 12]
 [24 25 12 12 12 12 12 12]
 [25 25 12 12 12 12

assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 0  1  0  0  1  1  1  0]
 [ 0  0  0  0  1  1  1  1]
 [ 1  0  1  0  1  1  1  1]
 [ 1  1  1  1  1  1  1  0]
 [ 1  0  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1]
 [ 1  0  1  1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  1  0  0  1  1 -1]
 [ 1  0  1  1  1  1  0 -1]
 [ 0  0  1  1  1  1  0 -1]
 [ 1  1  0  1  0  0  1 -1]
 [ 1  1  1  0  1  1  1 -1]
 [ 1  1  1  1  1  1  1 -1]
 [ 0  1  0  1  1  1  0 -1]
 [ 1  1  1  0  1  0  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
assigned max colors (eta_sites):
[[1 2 4 6 6 2 5 1]
 [1 2 1 5 6 6 2 2]
 [6 1 2 3 1 6 6 6]
 [3 3 1 1 6 5 6 6]
 [6 3 6 4 5 6 4 6]
 [6 6 3 6 5 2 6 5]
 [6 1 5 6 6 2 6 1]
 [6 6 2 6 6 5 4 2]]
clusters formed by bonds (eta_edge):
[[ 5  1  5  2  2  2  3  4]
 [ 5  5  5  2  6  7  4  4]
 [ 5  5  5  2  8  9 10 10]
 [11  5  2  2 12 12 12 13]
 [14 15 16 17 17 18 1

current no. cols (lambda): 6
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 1  1  1  0  0  1  1  0]
 [ 1  0  0  1  0  1  1  1]
 [ 1  1  0  1  1  1  1  1]
 [ 0  1  1  1  1  1  1  1]
 [ 0  1  1  0  1  0  1  1]
 [ 1  0  1  0  0  0  1  0]
 [ 1  1  1  1  1  1  0  0]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  1  0  1  0  1 -1]
 [ 1  1  0  0  0  1  1 -1]
 [ 1  1  0  0  1  1  0 -1]
 [ 1  0  1  1  0  1  1 -1]
 [ 1  1  1  1  1  0  0 -1]
 [ 1  1  1  0  1  1  0 -1]
 [ 0  0  1  1  0  1  1 -1]
 [ 1  1  1  1  1  1  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
assigned max colors (eta_sites):
[[2 4 6 6 3 6 6 6]
 [6 6 2 6 6 2 6 6]
 [5 5 1 3 6 5 6 6]
 [4 6 5 6 6 6 2 4]
 [3 4 5 6 6 1 3 6]
 [6 6 4 6 2 6 2 6]
 [2 6 6 6 3 6 6 5]
 [6 1 6 6 6 3 6 2]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  4  4  5  5  6]
 [ 7  8  4  4  4  4  9  6]
 [10  8  4  4  4 11 12 12]
 [13  4  4 14 15 15

 [4 6 6 1 6 6 4 3]]
clusters formed by bonds (eta_edge):
[[ 1  2  3  3  4  4  5  5]
 [ 6  7  4  4  4  4  8  8]
 [ 6  9  4  4  4  4  4 10]
 [11 11  4  4  4  4  4 12]
 [11 11  4  4  4  4  4 13]
 [14 15 16  4  4  4  4  4]
 [17 18  4  4  4 19 20  4]
 [21 18  4  4  4 19 19  4]]
config:
[[3 3 5 5 1 1 3 3]
 [1 4 1 1 1 1 2 2]
 [1 4 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 2]
 [1 1 1 1 1 1 1 3]
 [6 4 5 1 1 1 1 1]
 [4 1 1 1 1 3 3 1]
 [2 1 1 1 1 3 3 1]]

-------------------- iter 83 --------------------
probabilities for no. of colors (lambda): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
current no. cols (lambda): 6
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 1  1  1  1  1  0  1  1]
 [ 1  0  1  0  0  0  1  1]
 [ 1  1  1  1  0  0  0  1]
 [ 0  0  0  0  0  0  1  1]
 [ 1  1  1  1  0  1  0  1]
 [ 1  1  1  0  0  1  1  0]
 [ 1  1  0  0  0  0  0  1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  0  1  0  1  0 -1]
 [ 1  1  0  0  0  1

[[4 2 4 4 4 4 4 2]
 [4 6 4 4 1 4 5 1]
 [1 1 1 5 1 1 4 4]
 [1 1 1 1 1 1 4 2]
 [1 1 5 1 1 1 1 3]
 [1 3 1 1 1 1 1 1]
 [1 3 1 1 5 1 1 1]
 [1 2 1 1 1 2 1 1]]

-------------------- iter 92 --------------------
probabilities for no. of colors (lambda): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
current no. cols (lambda): 6
assigned no. cols (eta_lambda): 6
probabilities for edges: [0.63212056 0.36787944]
assigned bonds (eta_edges):
[[ 0  1  0  1  1  0  1  1]
 [ 1  1  1  1  0  1  1  1]
 [ 1  0  0  1  1  0  1  1]
 [ 0  1  1  0  0  0  1  1]
 [ 1  1  1  0  1  0  1  1]
 [ 0  0  0  1  1  0  0  0]
 [ 0  1  1  1  1  1  0  0]
 [-1 -1 -1 -1 -1 -1 -1 -1]]

[[ 1  1  1  1  0  0  1 -1]
 [ 1  1  1  1  1  1  1 -1]
 [ 0  0  1  1  0  1  0 -1]
 [ 1  0  0  0  0  1  1 -1]
 [ 1  1  1  0  0  1  1 -1]
 [ 1  1  1  0  1  0  0 -1]
 [ 1  1  0  1  1  1  0 -1]
 [ 1  1  0  0  1  1  1 -1]]
probabilities for colors (sites): [0.18126925 0.14841071 0.12150841 0.09948267 0.08144952 0.36787944]
assigned 